In [1]:
import math
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchtext

# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## Embedder 모듈

In [2]:
class Embedder(nn.Module):
    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()

        self.embeddings = nn.Embedding.from_pretrained(embeddings=text_embedding_vectors, freeze=True)

    def forward(self, x):
        x_vec = self.embeddings(x)

        return x_vec

In [3]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(max_length=256, batch_size=24)

batch = next(iter(train_dl))

net1 = Embedder(TEXT.vocab.vectors)

x = batch.Text[0]
x1 = net1(x)

print("입력 텐서 크기: ", x.shape)
print("출력 텐서 크기: ", x1.shape)

입력 텐서 크기:  torch.Size([24, 256])
출력 텐서 크기:  torch.Size([24, 256, 300])


## PositionalEncoder 모듈

In [6]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()

        self.d_model = d_model

        pe = torch.zeros(max_seq_len, d_model)

        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model,2):
                pe[pos,i] = math.sin(pos/ (10000 ** ((2*i)/d_model)))
                pe[pos,i+1] = math.cos(pos / (10000 ** ((2*i)/d_model)))

        self.pe = pe.unsqueeze(0)

        self.pe.requires_grad = False

    def forward(self, x):
        ret = math.sqrt(self.d_model) * x + self.pe
        return ret

In [7]:
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)

x = batch.Text[0]
x1 = net1(x)
x2 = net2(x1)

print("입력 텐서 크기: ", x1.shape)
print("출력 텐서 크기: ", x2.shape)

입력 텐서 크기:  torch.Size([24, 256, 300])
출력 텐서 크기:  torch.Size([24, 256, 300])
